# Explore ReadArray (.h5)

In [1]:
import os
import pandas as pd

In [2]:
!tree .. -L 1

..
├── README.md
├── dataset
├── docs
├── gtf
├── h5
└── videos

5 directories, 1 file


In [3]:
path_data = "../dataset/"
path_gtf = "../gtf/annotations.gtf.gz"

# sorted by read name (required for SEQC)
path_aligned_bam = os.path.join(path_data, "test_Aligned.out.bam")

# sorted by coordinate (required for pysam)
path_aligned_sorted_bam = os.path.join(path_data, "test_Aligned.out.sorted.bam")


path_h5 = os.path.join(path_data, "test.h5")

In [4]:
!ls "$path_data"

sort-bam.sh                     test_mini_summary.pdf
test.h5                         test_seqc_log.txt
test_Aligned.out.bam            test_sparse_counts_barcodes.csv
test_Aligned.out.sorted.bam     test_sparse_counts_genes.csv
test_Aligned.out.sorted.bam.bai test_sparse_molecule_counts.mtx
test_alignment_summary.txt      test_sparse_read_counts.mtx
test_cb-correction.csv.gz       test_summary.tar.gz
test_dense.csv


## Load ReadArray (*.h5)

In [5]:
from seqc.read_array import ReadArray

In [6]:
ra = ReadArray.load(path_h5)

In [7]:
ra

## Attributes

### ra.filter_codes

In [8]:
ra.filter_codes

{'no_gene': 1,
 'rmt_error': 2,
 'cell_error': 4,
 'low_polyt': 8,
 'gene_not_unique': 16,
 'primer_missing': 32,
 'lonely_triplet': 64}

### ra.data

In [9]:
ra.data

array([(1, 200983461452598, 38446774684, 0),
       (1, 232448540961013, 40377936093, 0),
       (0, 200570500499883, 41267181429, 0), ...,
       (1, 169038985719156, 33098951083, 0),
       (1, 231840668564334, 31598787309, 0),
       (1, 161332457818475, 32723551981, 0)],
      dtype=[('status', 'u1'), ('cell', '<i8'), ('rmt', '<i8'), ('n_poly_t', 'u1')])

In [10]:
len(ra.data)

252735

In [11]:
df_data = pd.DataFrame(ra.data)
df_data

,status,cell,rmt,n_poly_t
0,1,200983461452598,38446774684,0
1,1,232448540961013,40377936093,0
2,0,200570500499883,41267181429,0
3,1,231772364696924,41564723102,0
4,1,227499945647347,39153163702,0
...,...,...,...,...
252730,1,169082619287347,49237838683,0
252731,1,164701771590046,57689455348,0
252732,1,169038985719156,33098951083,0
252733,1,231840668564334,31598787309,0


### ra.genes

In [12]:
ra.genes

array([     0,      0, 225067, ...,      0,      0,      0], dtype=int32)

In [13]:
len(ra.genes)

252735

In [14]:
df_genes = pd.DataFrame(ra.genes, columns=["genes"])
df_genes

,genes
0,0
1,0
2,225067
3,0
4,0
...,...
252730,0
252731,0
252732,0
252733,0


### ra.positions

In [15]:
ra.positions

array([       0,        0, 15611873, ...,        0,        0,        0],
      dtype=int32)

In [16]:
len(ra.positions)

252735

In [17]:
df_pos = pd.DataFrame(ra.positions, columns=["pos"])
df_pos

,pos
0,0
1,0
2,15611873
3,0
4,0
...,...
252730,0
252731,0
252732,0
252733,0


## Merge

In [18]:
df_merged = pd.concat([df_data, df_genes, df_pos], axis=1)
df_merged

,status,cell,rmt,n_poly_t,genes,pos
0,1,200983461452598,38446774684,0,0,0
1,1,232448540961013,40377936093,0,0,0
2,0,200570500499883,41267181429,0,225067,15611873
3,1,231772364696924,41564723102,0,0,0
4,1,227499945647347,39153163702,0,0,0
...,...,...,...,...,...,...
252730,1,169082619287347,49237838683,0,0,0
252731,1,164701771590046,57689455348,0,0,0
252732,1,169038985719156,33098951083,0,0,0
252733,1,231840668564334,31598787309,0,0,0


## Translate Cell Barcode

_** IMPORTANT **_

SEQC encodes barcodes into numeric format

e.g. `GGGAGATTCACGGACC` --> `200983461452598`

In [19]:
from seqc.sequence.encodings import DNA3Bit

In [20]:
dna3bit = DNA3Bit()

In [21]:
cb = df_merged.cell.apply(lambda x: dna3bit.decode(x).decode())
cb

0         GGGAGATTCACGGACC
1         CACCAAAAGGACATCG
2         GGACGTCGTTCAACGT
3         CAGAGCCGTGCCCGTA
4         CTGCATCGTAAGCTCT
                ...       
252730    ACTATCTGTGTGTACT
252731    AGTAGCTCAGGCACTC
252732    ACTTCGCAGAACCGCA
252733    CAGGGCTAGATTGGGC
252734    AAGTGAATCGGAAGGT
Name: cell, Length: 252735, dtype: object

In [22]:
umi = df_merged.rmt.apply(lambda x: dna3bit.decode(x).decode())
umi

0         ATCTACCTACTA
1         AGACGGACATTG
2         ACTTGGCTTGCG
3         ACGGTACGTCTC
4         AATGGGTGCCCC
              ...     
252730    GGCCCTTAGGTT
252731    CGGCATCTGTCA
252732    TCCACCTCACGT
252733    TGTTTTCTGTGG
252734    TCTCTCTATTGG
Name: rmt, Length: 252735, dtype: object

In [23]:
df_merged = df_merged.assign(cb=cb)
df_merged = df_merged.assign(umi=umi)

In [24]:
df_merged

,status,cell,rmt,n_poly_t,genes,pos,cb,umi
0,1,200983461452598,38446774684,0,0,0,GGGAGATTCACGGACC,ATCTACCTACTA
1,1,232448540961013,40377936093,0,0,0,CACCAAAAGGACATCG,AGACGGACATTG
2,0,200570500499883,41267181429,0,225067,15611873,GGACGTCGTTCAACGT,ACTTGGCTTGCG
3,1,231772364696924,41564723102,0,0,0,CAGAGCCGTGCCCGTA,ACGGTACGTCTC
4,1,227499945647347,39153163702,0,0,0,CTGCATCGTAAGCTCT,AATGGGTGCCCC
...,...,...,...,...,...,...,...,...
252730,1,169082619287347,49237838683,0,0,0,ACTATCTGTGTGTACT,GGCCCTTAGGTT
252731,1,164701771590046,57689455348,0,0,0,AGTAGCTCAGGCACTC,CGGCATCTGTCA
252732,1,169038985719156,33098951083,0,0,0,ACTTCGCAGAACCGCA,TCCACCTCACGT
252733,1,231840668564334,31598787309,0,0,0,CAGGGCTAGATTGGGC,TGTTTTCTGTGG


# Filtering

In [25]:
# reads that are mapped to a gene (i.e. status=0)
x = df_merged[ df_merged.status == 0 ].head(n=1)
x

,status,cell,rmt,n_poly_t,genes,pos,cb,umi
2,0,200570500499883,41267181429,0,225067,15611873,GGACGTCGTTCAACGT,ACTTGGCTTGCG


In [26]:
cb = x.iloc[0].cb
cb

'GGACGTCGTTCAACGT'

In [27]:
umi = x.iloc[0].umi
umi

'ACTTGGCTTGCG'

In [28]:
gene_id = "ENSG{:011d}".format(x.iloc[0].genes)
gene_id

'ENSG00000225067'

## Look Up BAM

What if you want to find the raw sequence from the aligned BAM file?

### Quick and Dirty

In [29]:
!samtools view "$path_aligned_bam" | grep "$cb" | grep "$umi"

:GGACGTCGTTCAACGT:ACTTGGCTTGCG:;A00228:279:HFWFVDMXX:1:1470:21450:5102	0	chr19	15611873	255	24S67M	*	0	0	AGCAGTGGTATCAACGCAGAGTACATGGGATCATCAAGTTTCCGCTGACCACTGAGTCTGCCATGAAGAAGATAGAAGACAACAACACACT	FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFF:FFFFFFFFFFFFFFF	NH:i:1	HI:i:1	AS:i:61	nM:i:2


### Using Pysam

API Reference: https://pysam.readthedocs.io/en/latest/api.html#sam-bam-cram-files

In [30]:
import pysam

In [31]:
with pysam.AlignmentFile(path_aligned_sorted_bam, "rb") as bamfile:
    for read in bamfile.fetch():
        print(read)
        break

:AAAGTCCCACCAGGAA:GGAGCAGCCTTT:;A00228:279:HFWFVDMXX:1:1110:29731:27211	0	0	61360	255	91M	-1	-1	91	TCTCCTTCCAGTGAGGAAGCGGGGCCACCACCCAGCGTGTGCTCCATCTTTTCTGGCTGGGGAGAGGCCTTCATCTGCTGTAAAGGGTCCT	array('B', [37, 25, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 25, 37, 37, 37, 37, 25, 37, 37, 25, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 11, 37, 37, 37, 37, 11, 25, 37, 37, 37, 37, 37, 37, 37, 25, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 25])	[('NH', 1), ('HI', 1), ('AS', 89), ('nM', 0)]


In [32]:
with pysam.AlignmentFile(path_aligned_sorted_bam, "rb") as bamfile:
    for read in bamfile.fetch():
        if read.query_name.startswith(f":{cb}:{umi}"):
            print(read)
            break

:GGACGTCGTTCAACGT:ACTTGGCTTGCG:;A00228:279:HFWFVDMXX:1:1470:21450:5102	0	0	15611872	255	24S67M	-1	-1	67	AGCAGTGGTATCAACGCAGAGTACATGGGATCATCAAGTTTCCGCTGACCACTGAGTCTGCCATGAAGAAGATAGAAGACAACAACACACT	array('B', [37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 25, 37, 37, 37, 25, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37])	[('NH', 1), ('HI', 1), ('AS', 61), ('nM', 2)]


In [33]:
print(read.query_name)
print(read.query_sequence)
print(read.cigarstring)

:GGACGTCGTTCAACGT:ACTTGGCTTGCG:;A00228:279:HFWFVDMXX:1:1470:21450:5102
AGCAGTGGTATCAACGCAGAGTACATGGGATCATCAAGTTTCCGCTGACCACTGAGTCTGCCATGAAGAAGATAGAAGACAACAACACACT
24S67M


## Look Up GTF

What if you want to get more information about the gene from the annotation GTF file?

### Quick and Dirty

Below returns three rows (gene, transcript, and exon)

In [34]:
!gunzip -c "$path_gtf" | grep $gene_id

chr19	HAVANA	gene	15611657	15612122	.	+	.	gene_id "ENSG00000225067.4"; gene_type "processed_pseudogene"; gene_status "KNOWN"; gene_name "RPL23AP2"; level 1; tag "pseudo_consens"; havana_gene "OTTHUMG00000158039.3";
chr19	HAVANA	transcript	15611657	15612122	.	+	.	gene_id "ENSG00000225067.4"; transcript_id "ENST00000471227.3"; gene_type "processed_pseudogene"; gene_status "KNOWN"; gene_name "RPL23AP2"; transcript_type "processed_pseudogene"; transcript_status "KNOWN"; transcript_name "RPL23AP2-001"; level 1; ont "PGO:0000004"; tag "pseudo_consens"; tag "basic"; transcript_support_level "NA"; havana_gene "OTTHUMG00000158039.3"; havana_transcript "OTTHUMT00000350064.3";
chr19	HAVANA	exon	15611657	15612122	.	+	.	gene_id "ENSG00000225067.4"; transcript_id "ENST00000471227.3"; gene_type "processed_pseudogene"; gene_status "KNOWN"; gene_name "RPL23AP2"; transcript_type "processed_pseudogene"; transcript_status "KNOWN"; transcript_name "RPL23AP2-001"; exon_number 1; exon_id "ENSE00001824478.3";

### Using Pyranges

API Reference: https://pyranges.readthedocs.io/en/latest/autoapi/pyranges/readers/index.html#pyranges.readers.read_gtf

In [35]:
import pyranges as pr

In [36]:
gr = pr.read_gtf(path_gtf)

In [37]:
# literally pandas dataframe
gr

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_status,transcript_name,tag,transcript_support_level,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr19,HAVANA,gene,68402,69178,.,+,.,ENSG00000282542.1,TEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr19,HAVANA,transcript,68402,69178,.,+,.,ENSG00000282542.1,TEC,...,KNOWN,AC008993.2-001,basic,NA,OTTHUMT00000451405.4,NaN,NaN,NaN,NaN,NaN
2,chr19,HAVANA,exon,68402,69178,.,+,.,ENSG00000282542.1,TEC,...,KNOWN,AC008993.2-001,basic,NA,OTTHUMT00000451405.4,1,ENSE00003776314.1,NaN,NaN,NaN
3,chr19,HAVANA,gene,69166,69972,.,+,.,ENSG00000282798.1,TEC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr19,HAVANA,transcript,69166,69972,.,+,.,ENSG00000282798.1,TEC,...,KNOWN,LLNLR-222A1.1-001,basic,NA,OTTHUMT00000484821.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163196,chr19,HAVANA,exon,58597705,58597816,.,-,.,ENSG00000269600.1,antisense,...,KNOWN,AC016629.3-002,basic,3,OTTHUMT00000466989.1,2,ENSE00003217533.1,NaN,NaN,NaN
163197,chr19,HAVANA,exon,58593895,58594125,.,-,.,ENSG00000269600.1,antisense,...,KNOWN,AC016629.3-002,basic,3,OTTHUMT00000466989.1,3,ENSE00003183363.1,NaN,NaN,NaN
163198,chr19,HAVANA,transcript,58597182,58599355,.,-,.,ENSG00000269600.1,antisense,...,KNOWN,AC016629.3-001,basic,2,OTTHUMT00000466990.1,NaN,NaN,NaN,NaN,NaN
163199,chr19,HAVANA,exon,58599099,58599355,.,-,.,ENSG00000269600.1,antisense,...,KNOWN,AC016629.3-001,basic,2,OTTHUMT00000466990.1,1,ENSE00003187300.1,NaN,NaN,NaN


_** IMPORTANT **_

SEQC strips out the gene version number (e.g. suffix such as `.1`, `.4`, ...)

In [38]:
mask = (gr.gene_id == gene_id + ".4")
mask

31        False
32        False
33        False
34        False
35        False
          ...  
163196    False
163197    False
163198    False
163199    False
163200    False
Name: gene_id, Length: 163201, dtype: bool

In [39]:
gr[ mask ]

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_status,transcript_name,tag,transcript_support_level,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr19,HAVANA,gene,15611656,15612122,.,+,.,ENSG00000225067.4,processed_pseudogene,...,NaN,NaN,pseudo_consens,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr19,HAVANA,transcript,15611656,15612122,.,+,.,ENSG00000225067.4,processed_pseudogene,...,KNOWN,RPL23AP2-001,basic,NA,OTTHUMT00000350064.3,NaN,NaN,PGO:0000004,NaN,NaN
2,chr19,HAVANA,exon,15611656,15612122,.,+,.,ENSG00000225067.4,processed_pseudogene,...,KNOWN,RPL23AP2-001,basic,NA,OTTHUMT00000350064.3,1,ENSE00001824478.3,PGO:0000004,NaN,NaN


In [40]:
found = []

# to treat really like pandas dataframe, use the `.as_df()` method
for row in gr.as_df().itertuples():
    if gene_id in row.gene_id:
        found.append(row)
        
pd.DataFrame(found)

,Index,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,...,transcript_status,transcript_name,tag,transcript_support_level,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,25282,chr19,HAVANA,gene,15611656,15612122,.,+,.,ENSG00000225067.4,...,NaN,NaN,pseudo_consens,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25283,chr19,HAVANA,transcript,15611656,15612122,.,+,.,ENSG00000225067.4,...,KNOWN,RPL23AP2-001,basic,NA,OTTHUMT00000350064.3,NaN,NaN,PGO:0000004,NaN,NaN
2,25284,chr19,HAVANA,exon,15611656,15612122,.,+,.,ENSG00000225067.4,...,KNOWN,RPL23AP2-001,basic,NA,OTTHUMT00000350064.3,1,ENSE00001824478.3,PGO:0000004,NaN,NaN
